In [67]:
# !pip install -U -q PyDrive
 
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
 
 
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [68]:
pip install underthesea


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import  train_test_split
from sklearn.metrics import mean_squared_log_error
from underthesea import word_tokenize
import keras
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

dev_sents_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/dev/sents.txt",names=['sentence'],delimiter = "\t")
dev_sentiment_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/dev/sentiments.txt",names=['sentiment'],delimiter = "\t")

test_sents_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/test/sents.txt",names=['sentence'],delimiter = "\t")
test_sentiment_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/test/sentiments.txt",names=['sentiment'],delimiter = "\t")

train_sents_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/train/sents.txt",names=['sentence'],delimiter = "\t")
train_sentiment_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/_UIT-VSFC/train/sentiments.txt",names=['sentiment'],delimiter = "\t")

data_dev = {'sentence': dev_sents_df['sentence'],
            'sentiment':dev_sentiment_df['sentiment']}
data_test = {'sentence': test_sents_df['sentence'],
            'sentiment':test_sentiment_df['sentiment']}

data_train = {'sentence': train_sents_df['sentence'],
            'sentiment':train_sentiment_df['sentiment']}

df_dev = pd.DataFrame(data_dev)
df_train = pd.DataFrame(data_train)
df_test = pd.DataFrame(data_test)
df_dev

,sentence,sentiment
0,giáo trình chưa cụ thể .,0
1,giảng buồn ngủ .,0
2,"giáo viên vui tính , tận tâm .",2
3,"giảng viên nên giao bài tập nhiều hơn , chia n...",0
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...",0
...,...,...
1578,hướng dẫn lab mơ hồ .,0
1579,thầy cho chúng em những bài tập mang tính thực...,2
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0


# Mục mới

In [70]:
from keras.utils.np_utils import to_categorical
labels_train = to_categorical(df_train.sentiment, num_classes=3)
labels_test = to_categorical(df_test.sentiment, num_classes=3)
labels_dev = to_categorical(df_dev.sentiment, num_classes=3)

In [71]:
#convert sentiment to float label
df_train['sentiment']= labels_train
df_test['sentiment']= labels_test
df_dev['sentiment']= labels_dev

In [72]:
import string
def cleaneData(text):
    return text.translate(str.maketrans('','', string.punctuation))

In [73]:
## Remove Punc

clean_test_data = []
for i in range(0,len(df_test)):
    clean_test_data.append(cleaneData(df_test['sentence'][i]))

clean_train_data = []
for i in range(0,len(df_train)):
    clean_train_data.append(cleaneData(df_train['sentence'][i]))


clean_dev_data = []
for i in range(0,len(df_dev)):
    clean_dev_data.append(cleaneData(df_dev['sentence'][i]))

In [74]:
##Tokenize Word using Underthesea 
tokenize_arr=[]
for x in clean_train_data:
    tokenize_arr.append(word_tokenize(x))

for x in clean_test_data:
    tokenize_arr.append(word_tokenize(x))

for x in clean_dev_data:
    tokenize_arr.append(word_tokenize(x))

tokenize_arr

[['slide', 'giáo trình', 'đầy đủ'],
 ['nhiệt tình', 'giảng dạy', 'gần gũi', 'với', 'sinh viên'],
 ['đi', 'học', 'đầy đủ', 'full', 'điểm', 'chuyên', 'cần'],
 ['chưa',
  'áp dụng',
  'công nghệ thông tin',
  'và',
  'các',
  'thiết bị',
  'hỗ trợ',
  'cho',
  'việc',
  'giảng dạy'],
 ['thầy',
  'giảng',
  'bài',
  'hay',
  'có',
  'nhiều',
  'bài tập',
  'ví dụ',
  'ngay',
  'trên',
  'lớp'],
 ['giảng viên',
  'đảm bảo',
  'thời gian',
  'lên',
  'lớp',
  'tích cực',
  'trả lời',
  'câu hỏi',
  'của',
  'sinh viên',
  'thường xuyên',
  'đặt',
  'câu',
  'hỏi',
  'cho',
  'sinh viên'],
 ['em',
  'sẽ',
  'nợ',
  'môn',
  'này',
  'nhưng',
  'em',
  'sẽ',
  'học lại',
  'ở',
  'các',
  'học kỳ',
  'kế tiếp'],
 ['thời lượng',
  'học',
  'quá',
  'dài',
  'không',
  'đảm bảo',
  'tiếp thu',
  'hiệu quả'],
 ['nội dung',
  'môn học',
  'có',
  'phần',
  'thiếu',
  'trọng tâm',
  'hầu như',
  'là',
  'chung chung',
  'khái quát',
  'khiến',
  'sinh viên',
  'rất',
  'khó',
  'nắm',
  'được',
  '

In [75]:
words=[]
for m in range(0,len(tokenize_arr)):
    for n in range(0,len(tokenize_arr[m])):
        words.append(tokenize_arr[m][n])
df_Count = pd.DataFrame(words,columns=['word'])
df_Count['Num']= 1
df_GroupBy=df_Count.groupby('word').count()
df_GroupBy.sort_values('Num',ascending=False,inplace=True)
df_GroupBy

,Num
word,
thầy,4350
sinh viên,3738
dạy,3106
không,2920
và,2904
...,...
nhannh,1
nhiết,1
nhiếu,1


In [76]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vietnamese-stopwords.txt',names=['word'])
list_stopwords = data['word']
list_stopwords = np.asarray(list_stopwords)
def remove_stopword(text):
    text2=''
    for x in text:
        if x in list_stopwords:
            text2+=""
        else:
            text2+=x+ " "
    return text2
storage=[]
for x in range(0,len(tokenize_arr)):
    storage.append(remove_stopword(tokenize_arr[x]))
storage

['slide giáo trình đầy đủ ',
 'nhiệt tình giảng dạy gần gũi sinh viên ',
 'đi học đầy đủ full chuyên ',
 'áp dụng công nghệ thông tin thiết bị giảng dạy ',
 'thầy giảng bài tập ví dụ lớp ',
 'giảng viên lớp tích cực trả lời sinh viên thường xuyên câu sinh viên ',
 'nợ môn học lại học kỳ kế tiếp ',
 'thời lượng học tiếp thu hiệu quả ',
 'nội dung môn học trọng tâm hầu như khái quát sinh viên nắm nội dung môn học ',
 'trình bày bảng thay vì slide ',
 'thầy tận tình đi dạy ',
 'áp lực ',
 'dạy thầy wzjwz208 đi nước ngoài giữa chừng thầy wzjwz209 dạy thay ',
 'môn học nâng độ ',
 'sinh viên đi thăm quan công ty quy mô làm việc giúp vê ngành học ',
 'hài lòng ',
 'nhiệt tình vui vẻ ',
 'nhiệt tình trao đổi giúp đỡ học tập ',
 'phong cách giảng thầy gần gũi sinh viên ',
 'giảng dạy kết hợp hoạt động giải trí hiệu quả ',
 'seminar sinh viên sâu chuyên ngành môn học ',
 'tiết học thoải mái ',
 'thầy nhiệt tình sinh viên ',
 'giảng dạy nhiệt tình hướng dẫn chi tiết sinh viên ',
 'tiết tập chữa 

In [77]:
## Prepare data for training
x_train = pd.Series(clean_train_data)
y_train = pd.Series(df_train['sentiment'])

x_val = pd.Series(clean_dev_data)
y_val = pd.Series(df_dev['sentiment'])

x_test = pd.Series(clean_test_data)
y_test = pd.Series(df_test['sentiment'])

In [78]:
## Bulding vocabulary using skipgam and CBOW
def labelize_text_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result


all_x = pd.concat([x_train])
all_x_w2v = labelize_text_ug(all_x, 'all')

all_x_w2v

[TaggedDocument(words=['slide', 'giáo', 'trình', 'đầy', 'đủ'], tags=['all_0']),
 TaggedDocument(words=['nhiệt', 'tình', 'giảng', 'dạy', 'gần', 'gũi', 'với', 'sinh', 'viên'], tags=['all_1']),
 TaggedDocument(words=['đi', 'học', 'đầy', 'đủ', 'full', 'điểm', 'chuyên', 'cần'], tags=['all_2']),
 TaggedDocument(words=['chưa', 'áp', 'dụng', 'công', 'nghệ', 'thông', 'tin', 'và', 'các', 'thiết', 'bị', 'hỗ', 'trợ', 'cho', 'việc', 'giảng', 'dạy'], tags=['all_3']),
 TaggedDocument(words=['thầy', 'giảng', 'bài', 'hay', 'có', 'nhiều', 'bài', 'tập', 'ví', 'dụ', 'ngay', 'trên', 'lớp'], tags=['all_4']),
 TaggedDocument(words=['giảng', 'viên', 'đảm', 'bảo', 'thời', 'gian', 'lên', 'lớp', 'tích', 'cực', 'trả', 'lời', 'câu', 'hỏi', 'của', 'sinh', 'viên', 'thường', 'xuyên', 'đặt', 'câu', 'hỏi', 'cho', 'sinh', 'viên'], tags=['all_5']),
 TaggedDocument(words=['em', 'sẽ', 'nợ', 'môn', 'này', 'nhưng', 'em', 'sẽ', 'học', 'lại', 'ở', 'các', 'học', 'kỳ', 'kế', 'tiếp'], tags=['all_6']),
 TaggedDocument(words=['thời

In [79]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)

In [80]:
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 11426/11426 [00:00<00:00, 972209.95it/s]


In [81]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 11426/11426 [00:00<00:00, 967245.60it/s]


CPU times: user 7.29 s, sys: 99.6 ms, total: 7.39 s
Wall time: 4.67 s


In [82]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 11426/11426 [00:00<00:00, 1186357.99it/s]


In [83]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 11426/11426 [00:00<00:00, 1066592.13it/s]


CPU times: user 12.6 s, sys: 126 ms, total: 12.8 s
Wall time: 7.53 s


In [84]:
model_ug_cbow.save('w2v_model_ug_cbow.word2vec')
model_ug_sg.save('w2v_model_ug_sg.word2vec')

In [85]:
## Get key vector

model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')

In [86]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 1517 word vectors.


In [87]:
np.append(model_ug_cbow.wv['slide'],model_ug_sg.wv['slide'])

array([-2.61401176e-01,  1.62175107e+00,  1.03002715e+00,  6.26472533e-01,
        3.56188774e-01,  8.81690979e-01,  1.39163315e+00,  1.63929999e+00,
        2.04282641e+00, -4.91553098e-01, -1.11914372e+00, -1.00499189e+00,
        3.55138153e-01,  9.88042951e-01,  1.66749358e+00,  9.21200216e-01,
        2.75850922e-01,  4.74489033e-01, -6.06205821e-01,  3.21369588e-01,
        8.17125440e-01, -6.58772707e-01,  3.09582710e-01, -2.65843689e-01,
        1.39050841e+00,  7.36522436e-01,  6.96634203e-02, -7.59279430e-02,
        6.30199432e-01, -1.63601673e+00,  1.17159382e-01,  5.91402709e-01,
        1.55947328e-01, -9.05934870e-01, -6.28550708e-01, -7.20857605e-02,
        1.03883140e-01, -5.18135965e-01,  1.14236605e+00, -1.02921665e+00,
       -7.09874213e-01,  7.75876105e-01,  3.32618505e-01,  5.61442494e-01,
       -1.55701458e-01, -1.42371881e+00, -7.98617482e-01, -6.89395517e-02,
        2.52323198e+00, -3.42068344e-01,  8.17813873e-01,  7.18649745e-01,
       -5.36511898e-01,  

In [88]:
## Vectorize words into numberic (float)
tokenizer = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ ')
tokenizer.fit_on_texts(x_train)

sequences_digit = tokenizer.texts_to_sequences(x_train)

In [89]:
#Take a max length of vector (reason of all inout have to be in a same size of matrix)

length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

147

In [90]:
x_train_seq = pad_sequences(sequences_digit, maxlen=150)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (11426, 150)


In [91]:
sequences_val = tokenizer.texts_to_sequences(x_val)
x_val_seq = pad_sequences(sequences_val, maxlen=150)

In [92]:
print('Shape of data tensor:', x_val_seq.shape)

Shape of data tensor: (1583, 150)


In [93]:
num_words = 10000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [94]:
#Using Bert CNN to training model
# Validate on Dev dataset
labels_train = to_categorical(df_train.sentiment, num_classes=3)
labels_test = to_categorical(df_test.sentiment, num_classes=3)
labels_dev = to_categorical(df_dev.sentiment, num_classes=3)

In [95]:
from keras import backend as K
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


In [96]:
model_cnn = Sequential()
e = Embedding(10000, 200, weights=[embedding_matrix], input_length=150, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(3, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc',f1_m,precision_m, recall_m])
history_cnn=model_cnn.fit(x_train_seq, labels_train, validation_data=(x_val_seq, labels_dev), epochs=5, batch_size=40, verbose=2)

Epoch 1/5
286/286 - 17s - loss: 0.1909 - acc: 0.8835 - f1_m: 0.8805 - precision_m: 0.8801 - recall_m: 0.8819 - val_loss: 0.1290 - val_acc: 0.9305 - val_f1_m: 0.9296 - val_precision_m: 0.9278 - val_recall_m: 0.9316 - 17s/epoch - 60ms/step
Epoch 2/5
286/286 - 16s - loss: 0.1098 - acc: 0.9395 - f1_m: 0.9389 - precision_m: 0.9395 - recall_m: 0.9385 - val_loss: 0.1149 - val_acc: 0.9368 - val_f1_m: 0.9362 - val_precision_m: 0.9380 - val_recall_m: 0.9345 - 16s/epoch - 55ms/step
Epoch 3/5
286/286 - 16s - loss: 0.0793 - acc: 0.9596 - f1_m: 0.9591 - precision_m: 0.9586 - recall_m: 0.9597 - val_loss: 0.1292 - val_acc: 0.9324 - val_f1_m: 0.9315 - val_precision_m: 0.9314 - val_recall_m: 0.9317 - 16s/epoch - 55ms/step
Epoch 4/5
286/286 - 17s - loss: 0.0539 - acc: 0.9738 - f1_m: 0.9735 - precision_m: 0.9736 - recall_m: 0.9733 - val_loss: 0.1332 - val_acc: 0.9280 - val_f1_m: 0.9274 - val_precision_m: 0.9252 - val_recall_m: 0.9297 - 17s/epoch - 60ms/step
Epoch 5/5
286/286 - 16s - loss: 0.0383 - acc: 0.

In [97]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=150)

In [98]:
loss, accuracy, f1_score, precision, recall = model_cnn.evaluate(x_test_seq, labels_test, verbose=0)

In [100]:
print("Loss:"+ str(loss))
print("accuracy:"+ str(accuracy))
print("f1_score:"+ str(f1_score))
print("precision:"+ str(precision))
print("recall:"+ str(recall))


Loss:0.19349409639835358
accuracy:0.9156664609909058
f1_score:0.9153540134429932
precision:0.9129993319511414
recall:0.9177820086479187
